In [1]:
import graphlab as gl
from graphlab import aggregate as agg
import csv
from operator import itemgetter
from itertools import groupby
from collections import Counter
import sys
import numpy as np

In [2]:
test_user = gl.SFrame.read_csv("/home/elena/Desktop/PolimiRecSysChallenge2016/Dataset/test_set_xing.tsv", sep="\t")

2016-10-24 13:18:17,607 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.5 started. Logging: /tmp/graphlab_server_1477307895.log


Finished parsing file /home/elena/Desktop/PolimiRecSysChallenge2016/Dataset/test_set_xing.tsv

Parsing completed. Parsed 100 lines in 0.242238 secs.

This non-commercial license of GraphLab Create for academic use is assigned to elena.sacchi@mail.polimi.it and will expire on February 05, 2017.
------------------------------------------------------

Finished parsing file /home/elena/Desktop/PolimiRecSysChallenge2016/Dataset/test_set_xing.tsv

Parsing completed. Parsed 117130 lines in 0.080307 secs.


Inferred types from first line of file as 
column_type_hints=[int,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


In [3]:
%%capture
interactions = gl.SFrame.read_csv("/home/elena/Desktop/competition/data/interactions.csv", sep="\t")
users = gl.SFrame.read_csv("/home/elena/Desktop/competition/data/users.csv", column_type_hints=[int,str,str,str,str,str,str,str,str,str,str,str], sep="\t")
items = gl.SFrame.read_csv("/home/elena/Desktop/competition/data/items.csv", sep="\t")
items.rename({'id' : 'item_id'})
users.rename({'id' : 'user_id'})

user_id,jobroles,career_level,discipline_id,industry_id,country,region,experience_n_entries_class ...
4,2424638,3,0,8,de,10,2
7,"3287640,524375",0,3,8,de,1,1
9,3418410,2,0,0,de,0,3
13,3079492,NULL,0,0,de,2,1
19,3378061,4,0,0,de,0,2
23,"165098,1678893",3,0,1,de,1,3
24,"524518,3354924,1038436",NULL,0,0,de,2,3
24,"524518,3354924,1038436",NULL,0,0,de,2,3
28,"500718,944846",3,0,15,ch,0,2
32,"3341539,2629444",4,3,13,de,1,3


In [61]:
%%capture
impressions = gl.SFrame.read_csv("/home/elena/Desktop/competition/data/impressions.csv", sep="\t")

Cleaning
--

In [4]:
print len(users)
users = users.unique()
print len(users)

1500000
1367057


In [10]:
active_items = items[items['active_during_test']==1]
key = active_items['item_id']
values = active_items['active_during_test']
d = dict(zip(key,values))

In [13]:
test_user['items'] = test_user.apply(lambda x : x['items'].split(','))
test_user['in_set'] = test_user.apply(lambda x : [i for i in x['items'] if d.get(int(i))!=None])


In [15]:
test_user

user_id,items,in_set,items_len
9,"[348142, 1292778]","[348142, 1292778]",2
69,"[2477615, 26164, 154772,164464, 167939, 647613, ...","[2477615, 26164, 154772,164464, 167939, 647613, ...",47
118,"[89562, 679895, 1106820,1507745] ...","[89562, 679895, 1106820,1507745] ...",4
136,"[1834826, 172383]","[1834826, 172383]",2
174,"[49606, 484267, 810761,2113931, 1492371] ...","[49606, 484267, 810761,2113931, 1492371] ...",5
178,"[2204002, 465056]","[2204002, 465056]",2
185,[126842],[126842],1
193,[1085517],[1085517],1
235,"[2122651, 2694389,1652144, 1427393, ...","[2122651, 2694389,1652144, 1427393, ...",5
239,"[2751279, 2071627]","[2751279, 2071627]",2


In [17]:
test_user['items_len'] = test_user.apply(lambda x : len(x['items']))
test_user['in_set_len'] = test_user.apply(lambda x : len(x['in_set']))
test_user['diff'] = test_user.apply(lambda x : x['items_len']-x['in_set_len'])

In [18]:
len(test_user[test_user['diff']!=0])

1578

In [19]:
test_user[test_user['diff']!=0]

user_id,items,in_set,items_len,in_set_len,diff
242,"[2839001, 794156]",[794156],2,1,1
3806,"[422665, 2842008,1516671, 2791831, ...","[422665, 1516671,2791831, 1363769, ...",44,43,1
4297,"[749846, 2838664, 671237]","[749846, 671237]",3,2,1
6442,"[2542063, 2520632,2512110, 2844354, ...","[2542063, 2520632,2512110, 2444569, ...",51,48,3
8498,"[1848945, 1917838,2559475, 507992, 2589 ...","[1848945, 1917838,2559475, 507992, 2589 ...",21,20,1
12630,"[444353, 1233411,1284528, 1417525, ...","[444353, 1233411,1284528, 1417525, ...",30,29,1
18097,"[189231, 1889036,2244464, 6222, 1709172, ...","[189231, 1889036,2244464, 6222, 1709172, ...",12,11,1
18465,"[228922, 176252, 604267,365758, 321085, 1022181, ...","[228922, 176252, 604267,365758, 321085, 1022181, ...",19,18,1
19645,[2845804],[],1,0,1
20979,"[908875, 607985, 2837844,2237202, 2542791, ...","[908875, 607985, 2237202,2542791, 2032473, ...",60,59,1


In [21]:
test_user = test_user['user_id','in_set']
test_user.rename({'in_set' : 'items'})

user_id,items
9,"[348142, 1292778]"
69,"[2477615, 26164, 154772,164464, 167939, 647613, ..."
118,"[89562, 679895, 1106820,1507745] ..."
136,"[1834826, 172383]"
174,"[49606, 484267, 810761,2113931, 1492371] ..."
178,"[2204002, 465056]"
185,[126842]
193,[1085517]
235,"[2122651, 2694389,1652144, 1427393, ..."
239,"[2751279, 2071627]"


Test set
--

In [22]:
#cold start users:
test_user.export_csv("./cleaned_xing_test_set.csv",sep="\t",quote_level=csv.QUOTE_NONE)
cold_start = test_user.filter_by(interactions['user_id'],'user_id',exclude=True)
not_cold_start = test_user.filter_by(cold_start['user_id'],'user_id',exclude=True)

In [65]:
print len(test_user)
print len(cold_start)
print len(not_cold_start)

117130
38812
78318


In [80]:
#sample
cs = cold_start.sample(0.02505,seed=2016)
print len(cs)
ncs = not_cold_start.sample(0.1157,seed=2016)
print len(ncs)

1000
9000


In [45]:
#verify how many ncs have impressions:
print len(ncs.filter_by(impressions['user_id'],'user_id'))

NameError: name 'impressions' is not defined

In [48]:
(8884+492)/10000.
#in the original data it was 88%, so I think it's a good approx

0.9376

In [83]:
th_test_set = ncs.append(cs)
th_test_set = th_test_set.sort("user_id")
th_test_set['items'] = th_test_set.apply(lambda x : ",".join(x['items']))
th_test_set.export_csv("./test_set.csv",sep="\t",quote_level=csv.QUOTE_NONE)

In [52]:
interactions_ts = interactions.filter_by(th_test_set['user_id'],'user_id')

In [89]:
len(interactions_ts)/10000

23

Training set users
--

In [110]:
%%capture
users = gl.SFrame.read_csv("/home/elena/Desktop/competition/data/users.csv", column_type_hints=[int,str,str,str,str,str,str,str,str,str,str,str], sep="\t")
users.rename({'id':'user_id'})
users = users.unique()


In [120]:
print len(users)
users_tr = users.filter_by(th_test_set['user_id'],'user_id',exclude=True)
print len(users_tr)

1367057
1357057


In [121]:
#sample: 75% con interactions -> 30k -> 21k da prendere con
users_cs = users_tr.filter_by(interactions['user_id'],'user_id',exclude=True)
users_ncs = users_tr.filter_by(users_cs['user_id'],'user_id',exclude=True)
print len(users_tr)
print len(users_cs)
print len(users_ncs)

1357057
581370
775687


In [130]:
users_cs_tr = users_cs.sample(0.016)
print len(users_cs_tr) #9k
users_cs_tr = users_cs_tr[:9000]
users_ncs_tr = users_ncs.sample(0.028)
users_ncs_tr = users_ncs_tr[:21000]
print len(users_ncs_tr) #21k

9202
21000


In [131]:
test_users_prof = users.filter_by(th_test_set['user_id'],'user_id')

In [132]:
th_user_profile = users_cs_tr.append(users_ncs_tr).append(test_users_prof)
th_user_profile = th_user_profile.sort('user_id')

In [133]:
print len(th_user_profile.unique())
th_user_profile = th_user_profile.sort('user_id')
th_user_profile.export_csv("./user_profile.csv",sep="\t",quote_level=csv.QUOTE_NONE)

40000


In [134]:
#interactions:
i = interactions.filter_by(th_user_profile['user_id'],'user_id')

In [135]:
i = (i.groupby('user_id',operations={'nint' : agg.COUNT('item_id')}))

Items - interactions
--

In [137]:
interactions_th = interactions.filter_by(th_user_profile['user_id'],'user_id')
len(interactions_th)

474198

In [34]:
interactions_th = gl.SFrame.read_csv("interactions.csv", sep="\t")
items = gl.SFrame.read_csv("/home/elena/Desktop/competition/data/items.csv", sep="\t")
items.rename({'id' : 'item_id'})

Read 100 lines. Lines per second: 363.354

Finished parsing file /home/elena/Desktop/thesis/pumpkin-pie-solution/sampling/interactions.csv

Parsing completed. Parsed 100 lines in 0.277599 secs.

Finished parsing file /home/elena/Desktop/thesis/pumpkin-pie-solution/sampling/interactions.csv

Parsing completed. Parsed 474198 lines in 0.267022 secs.

...

Read 1295250 lines. Lines per second: 403300

Finished parsing file /home/elena/Desktop/competition/data/items.csv

Parsing completed. Parsed 1358098 lines in 3.55773 secs.


Inferred types from first line of file as 
column_type_hints=[int,str,int,int,int,str,int,str,str,int,str,str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


item_id,title,career_level,discipline_id,industry_id,country,region,latitude,longitude
19,"4298526,4316979,2643380",3,18,15,ch,0,47.4,8.5
116,822375,2,20,22,de,1,48.4,10.0
140,"232343,3582227",3,12,15,at,0,47.3,11.1
161,"2804344,1768488,1662103",3,15,21,de,0,48.1,16.9
190,3907924,3,14,15,de,6,53.6,10.0
244,"3772791,1938824",3,15,11,de,10,51.2,7.1
268,3582227,2,19,21,at,0,48.2,16.4
328,"3836325,2912912",3,12,15,de,10,50.9,6.9
344,,3,9,15,de,9,51.2,6.8
350,"2202904,76751",3,20,14,de,5,53.1,8.8


In [35]:
items_from_int = items.filter_by(interactions_th['item_id'],'item_id')

In [36]:
len(items_from_int)

202233

In [ ]:
#nel dataset della challenge, il 75% degli items era nelle interactions, il restante 25% no -> circa 66K

In [38]:
len(test_items)

44019

In [39]:
items_to_have = list(set(test_items))

In [40]:
items_test_set = items.filter_by(map(int,items_to_have),'item_id')
items_not_int = items_test_set.filter_by(items_from_int['item_id'],'item_id',exclude=True)

In [42]:
len(items_not_int)

21249

In [45]:
extra_items = items.filter_by(items_from_int['item_id'],'item_id',exclude=True)
extra_items = extra_items.filter_by(items_not_int['item_id'],'item_id',exclude=True)

In [46]:
extra_items = extra_items.sample(0.045,seed=2016)
len(extra_items)

50836

In [47]:
items_th = extra_items.append(items_from_int).append(items_not_int)

In [49]:
len(items_th)

274318

In [50]:
items_th.export_csv("./items.csv",sep="\t",quote_level=csv.QUOTE_NONE)
interactions_th.export_csv("./interactions.csv",sep="\t",quote_level=csv.QUOTE_NONE)

In [202]:
th_test_set = gl.SFrame.read_csv("test_set.csv",sep="\t")

Finished parsing file /home/elena/Desktop/thesis/pumpkin-pie-solution/sampling/test_set.csv

Parsing completed. Parsed 100 lines in 0.022131 secs.

Finished parsing file /home/elena/Desktop/thesis/pumpkin-pie-solution/sampling/test_set.csv

Parsing completed. Parsed 10000 lines in 0.015023 secs.

------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Impressions
--

In [63]:
impressions_th = impressions.filter_by(users['user_id'],'user_id')

In [64]:
len(impressions_th)

219700

In [65]:
impressions_th.export_csv("./impressions.csv",sep="\t",quote_level=csv.QUOTE_NONE)


Checking everything is ok
--

In [51]:
test_set =  gl.SFrame.read_csv("test_set.csv", sep="\t")
users =  gl.SFrame.read_csv("user_profile.csv", sep="\t",column_type_hints=[int,str,str,str,str,str,str,str,str,str,str,str])
items =  gl.SFrame.read_csv("items.csv", sep="\t")
interactions =  gl.SFrame.read_csv("interactions.csv", sep="\t")

Finished parsing file /home/elena/Desktop/thesis/pumpkin-pie-solution/sampling/test_set.csv

Parsing completed. Parsed 100 lines in 0.018789 secs.

Finished parsing file /home/elena/Desktop/thesis/pumpkin-pie-solution/sampling/test_set.csv

Parsing completed. Parsed 10000 lines in 0.015451 secs.

Finished parsing file /home/elena/Desktop/thesis/pumpkin-pie-solution/sampling/user_profile.csv

...

Parsing completed. Parsed 100 lines in 0.259793 secs.


Inferred types from first line of file as 
column_type_hints=[int,str,int,int,int,str,int,str,str,int,str,str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
------------------------------------------------------

Finished parsing file /home/elena/Desktop/thesis/pumpkin-pie-solution/sampling/interactions.csv

Parsing completed. Parsed 474198 lines in 0.246123 secs.


Inferred types from first line of file as 
column_type_hints=[int,int,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


In [8]:
print "number of test set users: ",len(test_set.unique())
print "number of total users: ",len(users.unique())
print "are test users in training? ", len(test_set), " = ", len(users.filter_by(map(str,test_set['user_id']),'user_id'))

number of test set users:  10000
number of total users:  40000
are test users in training?  10000  =  10000


In [25]:
print "total users with interactions", len(users.filter_by(interactions['user_id'],'user_id'))
print "test users with interactions", len(test_set.filter_by(interactions['user_id'],'user_id'))

total users with interactions 30000
test users with interactions 9000


In [26]:
print "total number of interactions", len(interactions)
print "avg number of interactions per user", len(interactions)/30000.

total number of interactions 474198
avg number of interactions per user 15.8066


In [52]:
print "total number of items", len(items)
print "total number of interacted items", len(items.filter_by(interactions['item_id'],'item_id'))

total number of items 274318
total number of interacted items 202233


In [53]:
test_items = ",".join(test_set['items']).split(",")
print "total number of test items is", len(test_items)," approx. ",len(test_items)/10000.,' items per user'
print "are all the test items in item.csv? ",len(list(set(test_items)))," == ",len(items.filter_by(map(int,test_items),'item_id'))

total number of test items is 44019  approx.  4.4019  items per user
are all the test items in item.csv?  31049  ==  31049


In [60]:
active = items[items['active_during_test']==1]
print "active items: ", len(active), "ratio", len(active)/float(len(items))
print "active interactions:", len(interactions.filter_by(active['item_id'],'item_id'))

active items:  89521 ratio 0.326340232868
active interactions: 150113


In [70]:
print "users without interactions",len(users.filter_by(interactions['user_id'],'user_id',exclude=True))
print "users without impressions",len(users.filter_by(impressions['user_id'],'user_id',exclude=True))
print "users with nothing",len(users.filter_by(impressions['user_id'],'user_id',exclude=True)\
                                      .filter_by(interactions['user_id'],'user_id',exclude=True))
print "test users with nothing",len(test_set.filter_by(impressions['user_id'],'user_id',exclude=True)\
                                      .filter_by(interactions['user_id'],'user_id',exclude=True))

users without interactions 10000
users without impressions 2079
users with nothing 948
test users with nothing 372


In [18]:
users = users['user_id','jobroles','career_level','discipline_id','industry_id','country','region','experience_n_entries_class'\
              ,'experience_years_experience','experience_years_in_current','edu_degree','edu_fieldofstudies']

In [20]:
users.export_csv('user_profile.csv',sep="\t",quote_level=csv.QUOTE_NONE)

In [14]:
users.column_names

<bound method SFrame.column_names of Columns:
	id	int
	jobroles	str
	career_level	str
	discipline_id	str
	industry_id	str
	country	str
	region	str
	experience_n_entries_class	str
	experience_years_experience	str
	experience_years_in_current	str
	edu_degree	str
	edu_fieldofstudies	str

Rows: 1500000

Data:
+----+------------------------+--------------+---------------+-------------+---------+
| id |        jobroles        | career_level | discipline_id | industry_id | country |
+----+------------------------+--------------+---------------+-------------+---------+
| 4  |        2424638         |      3       |       0       |      8      |    de   |
| 7  |     3287640,524375     |      0       |       3       |      8      |    de   |
| 9  |        3418410         |      2       |       0       |      0      |    de   |
| 13 |        3079492         |     NULL     |       0       |      0      |    de   |
| 19 |        3378061         |      4       |       0       |      0      |    de  